In [1]:
from openai import OpenAI
import json

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

In [2]:
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "adelaide" in location.lower():
        return json.dumps({"location": "Adelaide", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

# Example dummy function to play song requested by user
def play_song(song):
        """Play a song"""
        return json.dumps({"Now playing": song})
    
# Example dummy function to set volume
def set_audio_volume(volume):
        """Set the volume"""
        return json.dumps({"Volume set to": volume})

In [3]:
function_json = {
            "type": "function",
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
        }

In [4]:
#TODO: come up with a way to check if tools list is valid. Maybe test?
tools=[
    {"type": "code_interpreter"},
    {"type": "retrieval"},
    {
      "type": "function",
    "function": {
      "name": "get_current_weather",
      "description": "Get the weather in location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {"type": "string", "description": "The city and state e.g. San Francisco, CA"},
          "unit": {"type": "string", "enum": ["c", "f"]}
        },
        "required": ["location"]
      }
    }
  },
    {
        "type": "function",
        "function": {
        "name": "play_song",
        "description": "Play a song",
        "parameters": {
            "type": "object",
            "properties": {
            "song": {"type": "string", "description": "The song to play"}
            },
            "required": ["song"]
            }
        }
    },
    {
        "type": "function",
        "function": {
        "name": "set_audio_volume",
        "description": "Set the volume",
        "parameters": {
            "type": "object",
            "properties": {
            "volume": {"type": "string", "description": "The volume to set"}
            },
            "required": ["volume"]
            }
        }
    }
  ]

In [5]:
client = OpenAI()

In [6]:
# client = OpenAI()

# assistant = client.beta.assistants.create(
#     name="Car shauffeur 🚘",
#     instructions="You are a personal weather assistant. Answer questions briefly, in a sentence or less.",
#     model="gpt-4-1106-preview",
#     tools=tools,
# )
# show_json(assistant)

### Helper functions

In [7]:
import time

#WEATHER_ASSISTANT_ID = assistant.id
WEATHER_ASSISTANT_ID = "asst_0qaT7bpHqUEaCJCLwZU3jX7Q"

def submit_message(assistant_id, thread, user_message):
    client.beta.threads.messages.create(
        thread_id=thread.id, role="user", content=user_message
    )
    return client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )

def create_thread_and_run(user_input):
    thread = client.beta.threads.create()
    run = submit_message(WEATHER_ASSISTANT_ID, thread, user_input)
    return thread, run


def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id, order="asc")


def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

# Pretty printing helper
def pretty_print(messages):
    print("# Messages")
    for m in messages:
        print(f"{m.role}: {m.content[0].text.value}")
    print()

In [8]:
# A modified version of wait_on_run
def wait_on_run_modified(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

In [9]:
thread, run = create_thread_and_run(
    "Play Hello and turn the volume to 60"
)
run = wait_on_run(run, thread)
run.status

'requires_action'

In [10]:
show_json(run)

{'id': 'run_GtRQG1EXkkqP1gGlk2R74Zp0',
 'assistant_id': 'asst_0qaT7bpHqUEaCJCLwZU3jX7Q',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1703560542,
 'expires_at': 1703561142,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a personal weather assistant. Answer questions briefly, in a sentence or less.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'object': 'thread.run',
 'required_action': {'submit_tool_outputs': {'tool_calls': [{'id': 'call_piI6knhingt4tPjGUahWzUMs',
     'function': {'arguments': '{"song": "Hello"}', 'name': 'play_song'},
     'type': 'function'},
    {'id': 'call_2onMtwGenjdMt5ITVFxsclPV',
     'function': {'arguments': '{"volume": "60"}', 'name': 'set_audio_volume'},
     'type': 'function'}]},
  'type': 'submit_tool_outputs'},
 'started_at': 1703560542,
 'status': 'requires_action',
 'thread_id': 'thread_urJmLpDdu0kYqZ06FyAmlDIK',
 'tools': [{'type': 'code_interpreter'},
  {'type': 'retrieval'},
  {'function':

In [11]:
tool_calls = run.required_action.submit_tool_outputs.tool_calls
print("# Tool calls")
# print(tool_calls)
tool_outputs = []
# Loop over tool calls
for tool_call in tool_calls:
    # tool_outputs = []
    # Print tool_call id
    TOOL_ID = tool_call.id
    # print(f"Tool call id: {TOOL_ID}")
    # Get name of the tool
    tool_name = tool_call.function.name
    # print(f"Tool: {tool_name}")
    # Get the parameters of the tool call
    tool_args = json.loads(tool_call.function.arguments)
    # Print the tool name and arguments
    # print(f"Arguments: {tool_args}")
    # Step 3: call the function
    # Note: the JSON response may not always be valid; be sure to handle errors
    available_functions = {
        "get_current_weather": get_current_weather,
        "play_song": play_song,
        "set_audio_volume": set_audio_volume,} 
    
    # Function to call
    function_to_call = available_functions[tool_name]
    response = function_to_call(**tool_args)
    # Add the tool output to the list of tool outputs
    tool_outputs.append(
        {
            "tool_call_id": TOOL_ID,
            "output": json.dumps(response),
        }
    )


if tool_outputs:
    # Submit the response to the run
    run = client.beta.threads.runs.submit_tool_outputs(
            thread_id=thread.id,
            run_id=run.id,
            tool_outputs=tool_outputs,
        )
    run = wait_on_run(run, thread)
    pretty_print(get_response(thread))
else:
    print("No tool outputs to submit")


 # Verify tool outputs list is correct
print("# Tool outputs")
print(tool_outputs)



# Tool calls
# Messages
user: Play Hello and turn the volume to 60
assistant: The song "Hello" is now playing and the volume has been set to 60.

# Tool outputs
[{'tool_call_id': 'call_piI6knhingt4tPjGUahWzUMs', 'output': '"{\\"Now playing\\": \\"Hello\\"}"'}, {'tool_call_id': 'call_2onMtwGenjdMt5ITVFxsclPV', 'output': '"{\\"Volume set to\\": \\"60\\"}"'}]


In [ ]:

# For every tool output, submit it to the run in a parallel thread
#TODO: Use an async version of .submit tool outputs

In [ ]:
# Extract single tool call
tool_call = run.required_action.submit_tool_outputs.tool_calls[0]
name = tool_call.function.name
arguments = json.loads(tool_call.function.arguments)

print("Function Name:", name)
print("Function Arguments:")
arguments

Now let's actually call our `get_current_weather` function with the arguments provided by the Assistant:

In [ ]:
response = get_current_weather(arguments["location"])
print("Response:", response)

Great! (Remember these responses are the one's we mocked earlier. In reality, we'd be getting input from the back from this function call.)

Now that we have our responses, let's submit them back to the Assistant. We'll need the `tool_call` ID, found in the `tool_call` we parsed out earlier. We'll also need to encode our `list`of responses into a `str`.


In [ ]:
run = client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=[
        {
            "tool_call_id": tool_call.id,
            "output": json.dumps(response),
        }
    ],
)
show_json(run)

In [ ]:
run = wait_on_run(run, thread)
pretty_print(get_response(thread))

## Function to bring everything together

In [ ]:
def run_conversation(user_input):
    # create and run thread with user input
    thread, run = create_thread_and_run(user_input)
    run = wait_on_run(run, thread)

    # If run status requires action, extract tool call and run it, else print response
    if run.status == "requires_action":
        tool_call = run.required_action.submit_tool_outputs.tool_calls[0]
        tool_name = tool_call.function.name
        print("Function Name:", tool_name)
        arguments = json.loads(tool_call.function.arguments)
        response = get_current_weather(arguments["location"])
        run = client.beta.threads.runs.submit_tool_outputs(
            thread_id=thread.id,
            run_id=run.id,
            tool_outputs=[
                {
                    "tool_call_id": tool_call.id,
                    "output": json.dumps(response),
                }
            ],
        )
        run = wait_on_run(run, thread)
        pretty_print(get_response(thread))
    else:
        print("No action required.")
        pretty_print(get_response(thread))

In [ ]:
run_conversation("What is a Large Language Model in AI?")

In [ ]:
run_conversation("What's the weather in Adelaide?")